## **Cell 1: Install Dependencies**

In [1]:
# Install required packages
!pip install -q diffusers transformers accelerate imageio imageio-ffmpeg pillow gtts

print("✅ Installation complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.6 MB/s eta 0:00:00
✅ Installation complete!


## **Cell 2: Import Libraries**

In [2]:
import torch
from diffusers import DiffusionPipeline, AnimateDiffPipeline
import imageio
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from gtts import gTTS
from IPython.display import Video, Audio
import os

print(f"✅ GPU Available: {torch.cuda.is_available()}")
print(f"✅ GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


✅ GPU Available: True
✅ GPU Name: Tesla T4


## **Cell 3: Define Story**

In [15]:
# Your story in structured format
STORY = [
    {
        'scene': 1,
        'description': 'A cheerful classroom with a smart parrot wearing glasses sitting on a desk, colorful cartoon style',
        'narration': 'Magic School mein ek tota rehta tha jiska naam Tinku tha. Tinku bohat tez dimagh tha aur har problem ko ghour se dekh kar hal karta tha.'
    },
    {
        'scene': 2,
        'description': 'Children looking confused at a broken clock on the wall, bright classroom, animated style',
        'narration': 'Ek din class ki ghari achanak ruk gayi. Bachay hairan thay kyunke ghari kal bilkul theek chal rahi thi.'
    },
    {
        'scene': 3,
        'description': 'Parrot examining the clock closely, noticing loose screws, close-up cartoon view',
        'narration': 'Tinku ne ghari ko qareeb se dekha. Usne notice kiya ke ghari ke peechay ke screws dheelay thay, lekin ghari tooti hui nahi thi.'
    },
    {
        'scene': 4,
        'description': 'A cat holding a magnet near metal objects, playful scene, classroom corner',
        'narration': 'Tinku ki nazar billi Mimi par pari jo ek magnet ke sath khel rahi thi. Jahan jahan Mimi ja rahi thi, cheezen uski taraf kheench rahi thin.'
    },
    {
        'scene': 5,
        'description': 'Parrot connecting clues with thought bubbles, logical reasoning illustration',
        'narration': 'Tinku ne socha: agar magnet metal cheezon ko kheenchta hai, to ye ghari ke andar ke parts ko bhi rok sakta hai. Isi wajah se ghari band hui.'
    },
    {
        'scene': 6,
        'description': 'Children removing the magnet and fixing the clock, happy cartoon ending',
        'narration': 'Bachon ne magnet ko ghari se door rakha aur screws tight kar diye. Ghari dobara chalne lagi aur sab ne Tinku ki aqalmandi ki tareef ki.'
    }
]

print(f"✅ Story loaded with {len(STORY)} scenes")

✅ Story loaded with 6 scenes


In [3]:
"""# Your story in structured format
STORY = [
    {
        'scene': 1,
        'description': 'A young robot standing alone in a futuristic city at night, glowing lights, cartoon animation style',
        'narration': 'Shehar Neonville mein ek chhota robot Robo tha jo har raat akela sadkon par ghoomta tha. Uska khwab tha ke wo insano ki madad kar sake.'
    },
    {
        'scene': 2,
        'description': 'Broken street lights flickering, robot looking worried, dark but colorful environment',
        'narration': 'Ek raat achanak shehar ki lights band hone lagin. Robo ne dekha ke kai street lights toot chuki hain aur shehar andheray mein doob raha hai.'
    },
    {
        'scene': 3,
        'description': 'A suspicious shadowy figure running with wires, robot following, animated chase scene',
        'narration': 'Robo ne ek saaya dekha jo taarain chura kar bhaag raha tha. Usne himmat ki aur uske peechay bhaagna shuru kar diya.'
    },
    {
        'scene': 4,
        'description': 'Robot analyzing wires with glowing eyes, tech interface hologram, close-up view',
        'narration': 'Robo ne wires ko scan kiya aur pata chala ke ye shehar ke power system ka hissa hain. Agar ye na milti to shehar hamesha ke liye andheray mein reh jata.'
    },
    {
        'scene': 5,
        'description': 'Robot confronting a scared human boy holding wires, emotional cartoon scene',
        'narration': 'Robo ne dekha ke ek ghareeb larka wires chura raha tha taake apne ghar mein roshni kar sake. Larka bohat dara hua tha.'
    },
    {
        'scene': 6,
        'description': 'Robot helping the boy reconnect wires, city lights turning on, happy animated scene',
        'narration': 'Robo ne larkay ki madad ki aur wires wapas laga di. Shehar ki lights dobara jal utheen aur Robo pehli baar hero ban gaya.'
    }
]

print(f"✅ Story loaded with {len(STORY)} scenes")"""

✅ Story loaded with 6 scenes


In [4]:
"""# Your story in structured format
STORY = [
    {
        'scene': 1,
        'description': 'A detective standing in a dark room looking at a broken window, mysterious atmosphere, cartoon style',
        'narration': 'Ek din detective Ali ko pata chala ke school library se ek kimti kitab chori ho gayi hai. Usne dekha ke library ki khidki tooti hui thi.'
    },
    {
        'scene': 2,
        'description': 'Blue footprints on the floor leading to a door, detective following clues, animated style',
        'narration': 'Detective Ali ne zameen par neelay rang ke footprints dekhe. Ye footprints library se bahar playground ki taraf ja rahe thay.'
    },
    {
        'scene': 3,
        'description': 'A boy wearing a blue shirt hiding behind a tree, playground scene, colorful cartoon',
        'narration': 'Playground mein usne ek larka dekha jo neeli shirt pehna hua tha aur darakht ke peeche chhupa hua tha. Ye pehla shaq tha.'
    },
    {
        'scene': 4,
        'description': 'Detective examining a torn book page on the ground, close-up, dramatic lighting',
        'narration': 'Detective ne zameen par kitab ka ek phaata hua page dekha. Is page par kuch important information likhi hui thi.'
    },
    {
        'scene': 5,
        'description': 'Detective showing evidence to a group of students, classroom scene, bright colors',
        'narration': 'Detective Ali ne saray students ko ikatha kiya aur unhe saboot dikhaya. Usne pucha ke kiski shoes se neele rang ka paint lag sakta hai.'
    },
    {
        'scene': 6,
        'description': 'A boy confessing with his head down, emotional scene, warm lighting, cartoon style',
        'narration': 'Ek larka Ahmed aagay aaya aur usne qabool kiya ke usne kitab li thi kyunke wo padhna chahta tha. Detective ne usse samjhaya ke churana galat hai aur Ahmed ne maafi mangi.'
    }
]

print(f"✅ Story loaded with {len(STORY)} scenes")"""

'# Your story in structured format\nSTORY = [\n    {\n        \'scene\': 1,\n        \'description\': \'A detective standing in a dark room looking at a broken window, mysterious atmosphere, cartoon style\',\n        \'narration\': \'Ek din detective Ali ko pata chala ke school library se ek kimti kitab chori ho gayi hai. Usne dekha ke library ki khidki tooti hui thi.\'\n    },\n    {\n        \'scene\': 2,\n        \'description\': \'Blue footprints on the floor leading to a door, detective following clues, animated style\',\n        \'narration\': \'Detective Ali ne zameen par neelay rang ke footprints dekhe. Ye footprints library se bahar playground ki taraf ja rahe thay.\'\n    },\n    {\n        \'scene\': 3,\n        \'description\': \'A boy wearing a blue shirt hiding behind a tree, playground scene, colorful cartoon\',\n        \'narration\': \'Playground mein usne ek larka dekha jo neeli shirt pehna hua tha aur darakht ke peeche chhupa hua tha. Ye pehla shaq tha.\'\n    },\n  

## **Cell 4: Load Video Generation Model**

In [5]:
# Load text-to-video model
print("🔄 Loading video generation model (this may take 2-3 minutes)...")

try:
    # Try ModelScope text-to-video
    pipe = DiffusionPipeline.from_pretrained(
        "damo-vilab/text-to-video-ms-1.7b",
        torch_dtype=torch.float16,
        variant="fp16"
    )
    pipe.to("cuda")
    pipe.enable_model_cpu_offload()
    pipe.enable_vae_slicing()

    print("✅ Video model loaded successfully!")
    VIDEO_MODE = True

except Exception as e:
    print(f"⚠️ Video model failed: {e}")
    print("📸 Falling back to image generation mode")

    # Fallback to Stable Diffusion
    from diffusers import StableDiffusionPipeline

    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16
    )
    pipe.to("cuda")

    print("✅ Image model loaded!")
    VIDEO_MODE = False

🔄 Loading video generation model (this may take 2-3 minutes)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after Diffusers version 0.33.1. 


✅ Video model loaded successfully!


/usr/local/lib/python3.12/dist-packages/diffusers/pipelines/pipeline_utils.py:2186: FutureWarning: `enable_vae_slicing` is deprecated and will be removed in version 0.40.0. Calling `enable_vae_slicing()` on a `TextToVideoSDPipeline` is deprecated and this method will be removed in a future version. Please use `pipe.vae.enable_slicing()`.
  deprecate(


## **Cell 5: Generate Videos/Images for Each Scene**

In [16]:
import time

print("🎬 Starting scene generation...\n")

generated_videos = []

for i, scene_data in enumerate(STORY, 1):
    print(f"--- Scene {i}/{len(STORY)} ---")
    print(f"Description: {scene_data['description'][:50]}...")

    try:
        if VIDEO_MODE:
            # Generate video
            print("🎥 Generating video...")
            video_frames = pipe(
                scene_data['description'],
                num_inference_steps=25,
                num_frames=16,
                guidance_scale=7.5
            ).frames[0]

            # Save video
            output_path = f"scene_{i}.mp4"
            imageio.mimsave(output_path, video_frames, fps=8)

        else:
            # Generate image
            print("🎨 Generating image...")
            image = pipe(
                scene_data['description'],
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

            # Save image
            output_path = f"scene_{i}.png"
            image.save(output_path)

        generated_videos.append(output_path)
        print(f"✅ Scene {i} saved: {output_path}\n")

        # Delay to avoid memory issues
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error generating scene {i}: {e}\n")
        continue

print(f"\n✅ Generated {len(generated_videos)} scenes!")

🎬 Starting scene generation...

--- Scene 1/6 ---
Description: A cheerful classroom with a smart parrot wearing g...
🎥 Generating video...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Scene 1 saved: scene_1.mp4

--- Scene 2/6 ---
Description: Children looking confused at a broken clock on the...
🎥 Generating video...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Scene 2 saved: scene_2.mp4

--- Scene 3/6 ---
Description: Parrot examining the clock closely, noticing loose...
🎥 Generating video...


...

✅ Scene 4 saved: scene_4.mp4

--- Scene 5/6 ---
Description: Parrot connecting clues with thought bubbles, logi...
🎥 Generating video...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Scene 5 saved: scene_5.mp4

--- Scene 6/6 ---
Description: Children removing the magnet and fixing the clock,...
🎥 Generating video...


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Scene 6 saved: scene_6.mp4


✅ Generated 6 scenes!


## **Cell 6: Generate Audio Narration**

In [17]:
print("🎤 Generating audio narration...\n")

audio_files = []

for i, scene_data in enumerate(STORY, 1):
    print(f"Scene {i}: Generating audio...")

    try:
        # Use Google Text-to-Speech
        tts = gTTS(text=scene_data['narration'], lang='ur', slow=False)

        audio_path = f"narration_{i}.mp3"
        tts.save(audio_path)

        audio_files.append(audio_path)
        print(f"✅ Audio saved: {audio_path}")

    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Generated {len(audio_files)} audio files!")

🎤 Generating audio narration...

Scene 1: Generating audio...
✅ Audio saved: narration_1.mp3
Scene 2: Generating audio...
✅ Audio saved: narration_2.mp3
Scene 3: Generating audio...
✅ Audio saved: narration_3.mp3
Scene 4: Generating audio...
✅ Audio saved: narration_4.mp3
Scene 5: Generating audio...
✅ Audio saved: narration_5.mp3
Scene 6: Generating audio...
✅ Audio saved: narration_6.mp3

✅ Generated 6 audio files!


## **Cell 7: Preview First Scene**

In [18]:
# Preview first scene video
if generated_videos:
    print("🎬 Preview of Scene 1:")

    if VIDEO_MODE:
        Video(generated_videos[0], embed=True, width=600)
    else:
        from IPython.display import Image as IPImage
        display(IPImage(generated_videos[0]))

🎬 Preview of Scene 1:


## **Cell 8: Preview First Audio**

In [19]:
# Preview first narration
if audio_files:
    print("🎤 Preview of Scene 1 Narration:")
    Audio(audio_files[0], autoplay=False)

🎤 Preview of Scene 1 Narration:


## **Cell 9: Merge Videos + Audio (Install FFmpeg)**

In [20]:
# Install moviepy for merging
!pip install -q moviepy

from moviepy.editor import VideoFileClip, ImageClip, AudioFileClip, concatenate_videoclips

print("🎬 Merging scenes with audio...\n")

final_clips = []

for i in range(len(generated_videos)):
    try:
        print(f"Processing scene {i+1}...")

        # Load audio
        audio = AudioFileClip(audio_files[i])
        audio_duration = audio.duration

        if VIDEO_MODE:
            # Load video
            video = VideoFileClip(generated_videos[i])

            # Loop video to match audio length
            num_loops = int(audio_duration / video.duration) + 1
            video = concatenate_videoclips([video] * num_loops)
            video = video.subclip(0, audio_duration)

        else:
            # Convert image to video
            video = ImageClip(generated_videos[i]).set_duration(audio_duration)

        # Add audio
        final_clip = video.set_audio(audio)
        final_clips.append(final_clip)

        print(f"✅ Scene {i+1} merged ({audio_duration:.1f}s)")

    except Exception as e:
        print(f"❌ Error merging scene {i+1}: {e}")

print(f"\n✅ {len(final_clips)} clips ready for final merge!")

🎬 Merging scenes with audio...

Processing scene 1...
✅ Scene 1 merged (11.6s)
Processing scene 2...
✅ Scene 2 merged (8.8s)
Processing scene 3...
✅ Scene 3 merged (11.3s)
Processing scene 4...
✅ Scene 4 merged (12.5s)
Processing scene 5...
✅ Scene 5 merged (13.2s)
Processing scene 6...
✅ Scene 6 merged (12.2s)

✅ 6 clips ready for final merge!


## **Cell 10: Create Final Video**

In [21]:
print("🎞️ Creating final video...\n")

# Merge all clips
final_video = concatenate_videoclips(final_clips, method="compose")

# Export
output_file = "final_mystery_story.mp4"

final_video.write_videofile(
    output_file,
    fps=24,
    codec='libx264',
    audio_codec='aac',
    bitrate='3000k'
)

print(f"\n✅ Final video created: {output_file}")
print(f"⏱️ Total duration: {final_video.duration / 60:.2f} minutes")

🎞️ Creating final video...

Moviepy - Building video final_mystery_story.mp4.
MoviePy - Writing audio in final_mystery_storyTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_mystery_story.mp4



t:  22%|██▏       | 373/1669 [00:04<00:19, 67.88it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file scene_2.mp4, 196608 bytes wanted but 0 bytes read,at frame 15/17, at time 1.88/2.00 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  35%|███▍      | 580/1669 [00:07<00:18, 58.84it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file scene_3.mp4, 196608 bytes wanted but 0 bytes read,at frame 15/17, at time 1.88/2.00 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  51%|█████     | 854/1669 [00:13<00:30, 26.65it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file scene_4.mp4, 196608 bytes wanted but 0 bytes read,at frame

Moviepy - Done !
Moviepy - video ready final_mystery_story.mp4

✅ Final video created: final_mystery_story.mp4
⏱️ Total duration: 1.16 minutes


## **Cell 11: Download Final Video**

In [22]:
# Download to your computer
from google.colab import files

print("📥 Downloading final video...")
files.download(output_file)

print("✅ Download started! Check your browser's download folder.")

📥 Downloading final video...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started! Check your browser's download folder.


## **Method 1: Save to Google Drive (Best)**
## **Step 1: Mount Google Drive **

In [13]:
# Cell 1: Mount Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted!")

ValueError: mount failed

## **Step 2: Download & Save Model**

In [ ]:
# Cell 2: Load and Save Model
from diffusers import DiffusionPipeline
import torch

# Define save path in Google Drive
MODEL_PATH = "/content/drive/MyDrive/AI_Models/text_to_video"

# Check if model already exists
import os
if os.path.exists(MODEL_PATH):
    print("✅ Loading model from Google Drive (fast)...")
    pipe = DiffusionPipeline.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16
    )
else:
    print("📥 Downloading model (one-time, 2-3 min)...")
    pipe = DiffusionPipeline.from_pretrained(
        "damo-vilab/text-to-video-ms-1.7b",
        torch_dtype=torch.float16,
        variant="fp16"
    )

    # Save to Google Drive
    print("💾 Saving model to Google Drive...")
    pipe.save_pretrained(MODEL_PATH)
    print("✅ Model saved!")

# Move to GPU
pipe.to("cuda")
print("✅ Model ready to use!")

# ===== QUICK SETUP WITH AUTO-SAVE =====

In [ ]:
# ===== QUICK SETUP WITH AUTO-SAVE =====

from google.colab import drive
drive.mount('/content/drive')

from diffusers import DiffusionPipeline
import torch
import os

MODEL_PATH = "/content/drive/MyDrive/AI_Models/video_model"

# Auto load/save
if os.path.exists(MODEL_PATH):
    pipe = DiffusionPipeline.from_pretrained(MODEL_PATH, torch_dtype=torch.float16)
    print("✅ Loaded from Drive (instant)")
else:
    pipe = DiffusionPipeline.from_pretrained(
        "damo-vilab/text-to-video-ms-1.7b",
        torch_dtype=torch.float16,
        variant="fp16"
    )
    os.makedirs(MODEL_PATH, exist_ok=True)
    pipe.save_pretrained(MODEL_PATH)
    print("✅ Downloaded & saved")

pipe.to("cuda")
print("🚀 Ready!")